In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import time
# 查询系统可用的 GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# 确保有可用的 GPU 如果没有, 则会报错
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# 设置参数,该段务必在运行jupyter的第一段代码执行，否则会无法初始化成功
# 仅在需要时申请显存空间（程序初始运行时消耗很少的显存，随着程序的运行而动态申请显存）
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
![赛题评估指标](./markdown_pics/赛题评估指标.png)

#### 1.赛题背景

![赛题背景--涂鸦图像识别](./markdown_pics/赛题背景--涂鸦图像识别.png)

![赛题评估指标](./markdown_pics/赛题评估指标.png)

##### 1.这个公式的第一部分就是求平均值，也就是M代表的是Mean，那么主要需要研究AP（Average Precision）的计算方式

- 由于预测的结果在本图中需要预测三个结果，如果模型仅仅预测出了两个结果，那么 min(2,3) = 2, 如果模型预测了4个，那么就是min(4,3) =3 ,也就是说，只取最多3个结果
- 那么第二部分如果min(n,3) = 3,那么ΣP(k) = P(1) + P(2) + P(3)

##### 用代码实现一下这个公式，方便加深理解

In [2]:
def apk(actual, predicted, k = 10):
    # 这里在求min函数部分
    if len(predicted) > k :
        predicted = predicted[:k]

    # 初始化
    score = 0.0
    num_hits = 0.0

    # 对预测值的结果分别将序号和预测值做键值对
    # 遍历预测值的顺序和值
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / ( i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


##### 真实值 1
##### 预测值 1,2,3,4,5

##### 取 k = 3 
##### 预测值截断取 1,2,3

##### k = 1, 则 真实值1 == 预测值1，P(1) = 1
##### k = 2, 则 真实值1 != 预测值2，P(2) = 0, 若真实值1 != 预测值1，那么P(2) = 1/(1+1) = 0.5
##### k = 3, 则 真实值1 != 预测值3，P(3) = 0
##### 因此，总分 =P(1) + P(2) + P(3) =  1 + 0 + 0 = 1
##### 然后取平均值= 0.33333

In [3]:
actual = [1]
predicted = [1,2,3,4,5]

print('Answer = ', actual, 'predicted = ', predicted)
print('AP@5 = ', apk(actual, predicted, 5) )

Answer =  [1] predicted =  [1, 2, 3, 4, 5]
AP@5 =  1.0


In [4]:
predicted = [2,1,3,4,5]
　
print('Answer = ', actual, 'predicted = ', predicted)
print('AP@5 = ', apk(actual, predicted, 5) )

Answer =  [1] predicted =  [2, 1, 3, 4, 5]
AP@5 =  0.5


In [5]:
predicted = [3,2,1,4,5]

print('Answer = ', actual, 'predicted = ', predicted)
print('AP@5 = ', apk(actual, predicted, 5) )

Answer =  [1] predicted =  [3, 2, 1, 4, 5]
AP@5 =  0.3333333333333333


In [6]:
predicted = [4,2,3,1,5]

print('Answer = ', actual, 'predicted = ', predicted)
print('AP@5 = ', apk(actual, predicted, 5) )

Answer =  [1] predicted =  [4, 2, 3, 1, 5]
AP@5 =  0.25


In [7]:
predicted = [4,2,3,5,1]

print('Answer = ', actual, 'predicted = ', predicted)
print('AP@5 = ', apk(actual, predicted, 5) )

Answer =  [1] predicted =  [4, 2, 3, 5, 1]
AP@5 =  0.2


In [10]:
actual = [1,2,3]
predicted = [4,2,3,5,1]

print('Answer = ', actual, 'predicted = ', predicted)
print('AP@5 = ', apk(actual, predicted, 5) )

Answer =  [1, 2, 3] predicted =  [4, 2, 3, 5, 1]
AP@5 =  0.5888888888888889


由此来看，这个序列中，真实值和预测值的序列越接近，那么分数越高

#### 2.数据解读

![draw数据格式解读](./markdown_pics/draw数据格式解读.png)

![draw数据读取示例](./markdown_pics/draw数据读取示例.png)

##### 原始数据集和简化版数据集在drawing上的数据存储不一致

![draw数据读取示例2](./markdown_pics/draw数据读取示例2.png)

![draw数据字段说明](./markdown_pics/draw数据字段说明.png)

![draw一行包含一个图形](./markdown_pics/draw一行包含一个图形.png)

##### 下图的解释
- 第一笔，x,y表示绘画的坐标，t表示毫秒单位，比如第一个像素的坐标[[x0,y0]]，时间为t0，以此类推
- 第二笔，同上理解

![对一个draw图进行数据拆分](./markdown_pics/对一个draw图进行数据拆分.png)

![原始数据和简化版数据的区别](./markdown_pics/原始数据和简化版数据的区别.png)

![draw数据EDA](./markdown_pics/draw数据EDA.png)


![draw数据EDA2](./markdown_pics/draw数据EDA2.png)

##### 下图的可视化，一种可视化为了黑白图片，另外可视化成了RGB彩色图片

![draw可视化预览一下图形](./markdown_pics/draw可视化预览一下图形.png)


#### 3.思路分析

![draw建模思路分析](./markdown_pics/draw建模思路分析.png)

#### 4.模型构建

![draw模型构建思路](./markdown_pics/draw模型构建思路.png)

![预训练模型选择](./markdown_pics/预训练模型选择.png)